## K-Nearest Neighbors Classification from Scratch with NumPy

K-nearest neighhbors (KNN) is a machine learning algorithm that can be used both for classification and regression purposes. It falls under the category of supervised learning algorithms that predicts target values for unseen observations. In other words, it operates on labelled datasets and predicts either a class (classification) or a numeric value (regression) for the test data.

In this post, I will be implementing KNN classification algorithm from scratch using NumPy library only. As I have mentioned in my previous posts, I won't be using 
